In [1]:
import sys
sys.path.append("..")

from utils import data_dir, model_dir

import os
import random
import numpy as np
import time

import joblib

import tqdm
from pprint import pprint
import matplotlib as mpl
import matplotlib.pyplot as plt
parameters = {'axes.labelsize': 12,
          'xtick.labelsize': 12,
          'ytick.labelsize': 12,
          'legend.fontsize': 12,
          'lines.linewidth' : 2,
          'lines.markersize' : 7}
plt.rcParams.update(parameters)

In [2]:
import seaborn as sns
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, confusion_matrix, ConfusionMatrixDisplay, auc, precision_recall_curve
from sklearn.preprocessing import label_binarize

In [3]:
task_name = "scalar1"

In [4]:
def findBin(bins, var, ibin):
    for i in range(len(bins)-1):
        if var >= bins[i] and var < bins[i+1]:
            ibin.append(i)

In [5]:
# get root files and convert them to array
branch_labels = {"frac_first": "$f_{1}$",
                 "first_lateral_width_eta_w20": "$w_{s20}$",
                 "first_lateral_width_eta_w3": "$w_{s3}$",
                 "first_fraction_fside": "$f_{side}$",
                 "first_dEs": "$\Delta E_{s}$",
                 "first_Eratio": "$E_{ratio}$",
                 "second_R_eta": "$R_{\eta}$",
                 "second_R_phi": "$R_{\phi}$",
                 "second_lateral_width_eta_weta2": "$w_{\eta2}$"
                 }
branch_names = list(branch_labels.keys())
pprint(branch_names)
branch_ene = """total_e""".split(",")

['frac_first',
 'first_lateral_width_eta_w20',
 'first_lateral_width_eta_w3',
 'first_fraction_fside',
 'first_dEs',
 'first_Eratio',
 'second_R_eta',
 'second_R_phi',
 'second_lateral_width_eta_weta2']


In [6]:
bg0Legend = "$\gamma$"
bg1Legend = "$\pi^0$"
sigLegend = {
    "scalar1": r"$h_2\rightarrow\pi^0\pi^0$",
    "axion1": r"$a\rightarrow\gamma\gamma$",
    "axion2": r"$a\rightarrow3\pi^0$"
}[task_name]

In [7]:
def as_matrix(tree, columns):
    """
    tree is an npz object containing string keys (columns) and np.array values    """
    return np.stack([tree[col] for col in columns])


In [8]:
n_train = 70000
signal0_tree = np.load(f"{data_dir}/processed/bdt_vars/{task_name}_bdt_vars.npz")
signal0 = as_matrix(signal0_tree, columns=branch_names).T
signal0_ene = as_matrix(signal0_tree, columns=branch_ene).T
train_signal0 = signal0[:n_train]
test_signal0 = signal0[n_train:]
train_signal0_ene = signal0_ene[:n_train]
test_signal0_ene = signal0_ene[n_train:]

background0_tree = np.load(f"{data_dir}/processed/bdt_vars/gamma_bdt_vars.npz")
background0 = as_matrix(background0_tree, columns=branch_names).T
background0_ene = as_matrix(background0_tree, columns=branch_ene).T
train_background0 = background0[:n_train]
test_background0 = background0[n_train:]
train_background0_ene = background0_ene[:n_train]
test_background0_ene = background0_ene[n_train:]

background1_tree = np.load(f"{data_dir}/processed/bdt_vars/pi0_bdt_vars.npz")
background1 = as_matrix(background1_tree, columns=branch_names).T
background1_ene = as_matrix(background1_tree, columns=branch_ene).T
train_background1 = background1[:n_train]
test_background1 = background1[n_train:]
train_background1_ene = background1_ene[:n_train]
test_background1_ene = background1_ene[n_train:]

## Generate input histogram plots

In [9]:
def plot_inputs(outdir, vars, branch_labels, sig, sig_w, bkg, bkg_w, bkg2, bkg2_w, sigLegend, bg0Legend, bg1Legend):    
    for n, var in enumerate(vars):
        _, bins = np.histogram(np.concatenate(
            (sig[:, n], bkg[:, n], bkg2[:, n])), bins=40)
        sns.distplot(sig[:, n], hist_kws={'weights': sig_w}, bins=bins, kde=False,
                     norm_hist=True, color='orange', label='{}'.format(sigLegend))
        sns.distplot(bkg[:, n], hist_kws={'weights': bkg_w}, bins=bins,
                     kde=False, norm_hist=True, color='b', label='{}'.format(bg0Legend))
        sns.distplot(bkg2[:, n], hist_kws={'weights': bkg2_w}, bins=bins,
                     kde=False, norm_hist=True, color='g', label='{}'.format(bg1Legend))
        
        plt.legend()
        if var == "first_dEs":
            plt.subplots_adjust(left=0.15)
        plt.xlabel('{}'.format(branch_labels[var]), loc='right', fontsize=38)
        plt.ylabel('Entries', fontsize=24)

        if var in ["second_lateral_width_eta_weta2", "first_dEs"]:
            plt.xticks(fontsize=17, rotation=45)
        else:
            plt.xticks(fontsize=17)
        plt.yticks(fontsize=17)
        # https://stackoverflow.com/questions/42281851/how-to-add-padding-to-a-plot-in-python
        plt.tight_layout()
        
        plt.savefig(os.path.join(outdir, 'input_{}.pdf'.format(var)))
        plt.close()

In [10]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)

    output_dir = f"./gbdt_results_9var/{task_name}"
    os.makedirs(output_dir, exist_ok=True)
    plot_inputs(output_dir, branch_names, branch_labels, train_signal0,
                None, train_background0, None, train_background1, None, sigLegend, bg0Legend, bg1Legend)

/tmp/ipykernel_15050/4272367960.py:5: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(sig[:, n], hist_kws={'weights': sig_w}, bins=bins, kde=False,
/tmp/ipykernel_15050/4272367960.py:7: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(bkg[:, n], hist_kws={'weights': bkg_w}, bins=bins,
/tmp/ipykernel_

## Train the BDT

In [11]:
train_X_raw = np.concatenate(
    (train_signal0, train_background0, train_background1))
train_X_raw_ene = np.concatenate(
    (train_signal0_ene, train_background0_ene, train_background1_ene))
test_X_raw = np.concatenate(
    (test_signal0, test_background0, test_background1))
test_X_raw_ene = np.concatenate(
    (test_signal0_ene, test_background0_ene, test_background1_ene))

In [12]:
processLabels = {sigLegend: 2, bg0Legend: 0, bg1Legend: 1}
processColumns = [bg0Legend, sigLegend, bg1Legend]
iColForSig = 1

In [13]:
sortedLabels = []
for key in processLabels:
    sortedLabels.append(processLabels[key])
sortedLabels.sort()

train_y_raw = np.concatenate((np.zeros(train_signal0.shape[0])+processLabels[sigLegend], np.zeros(
    train_background0.shape[0])+processLabels[bg0Legend], np.zeros(train_background1.shape[0])+processLabels[bg1Legend]))
test_y_raw = np.concatenate((np.zeros(test_signal0.shape[0])+processLabels[sigLegend], np.zeros(
    test_background0.shape[0])+processLabels[bg0Legend], np.zeros(test_background1.shape[0])+processLabels[bg1Legend]))

print(len(train_signal0))
print(len(test_signal0))

print('part2')
for key in processLabels:
    print("Length for", key, "is", len(
        test_y_raw[test_y_raw == processLabels[key]]))

70000
30000
part2
Length for $h_2\rightarrow\pi^0\pi^0$ is 30000
Length for $\gamma$ is 30000
Length for $\pi^0$ is 30000


In [14]:
X_train = train_X_raw
# https://datascience.stackexchange.com/questions/11928/valueerror-input-contains-nan-infinity-or-a-value-too-large-for-dtypefloat32
print("check X_train NaN")
np.where(np.isnan(X_train))
print("check X_train Inf")
np.where(np.isinf(X_train))
# print("Replace X_train")
# X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
# X_train = np.nan_to_num(X_train.astype(np.float32))

X_test = test_X_raw
X_test_ene = test_X_raw_ene
# X_test_comb = list(zip(X_test, X_test_ene))
# print("X_test_comb", X_test_comb)

y_train = train_y_raw.astype(int)
y_test = test_y_raw.astype(int)

check X_train NaN
check X_train Inf


In [15]:
# May as well save this data for convenience
os.makedirs(f"{data_dir}/processed/bdt", exist_ok=True)
np.save(f"{data_dir}/processed/bdt/{task_name}_X_train", X_train)
np.save(f"{data_dir}/processed/bdt/{task_name}_y_train", y_train)
np.save(f"{data_dir}/processed/bdt/{task_name}_X_test", X_test)
np.save(f"{data_dir}/processed/bdt/{task_name}_y_test", y_test)

In [16]:
# Takes ~5 minutes on four GeForce RTX 2080 Tis
print('training')
from pprint import pprint

n_boost_stages = 100

def fmt_seconds(s):
    mins = int(s / 60)
    secs = int(s % 60)
    return f"{mins:02d}:{secs:02d}"

def monitor(i, self, locals):
    global loss_func, start_time

    raw_preds = locals["raw_predictions"]
    loss = locals["loss_"](y_train, raw_preds)
    y_pred = np.argmax(raw_preds, axis=1)
    acc = np.mean(y_train == y_pred)
    ETA = (n_boost_stages - i - 1) * (time.time() - start_time) / (i + 1)
    print(f"Step {i+1:>3}/{n_boost_stages} - loss: {loss:.5f}, acc: {acc:.5f} | ETA: {fmt_seconds(ETA)}")

# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
bdt = GradientBoostingClassifier(
    max_depth=5,
    min_samples_leaf=200,
    min_samples_split=10,
    n_estimators=n_boost_stages,
    learning_rate=0.5,
    warm_start=True
)

start_time = time.time()
bdt.fit(X_train, y_train, monitor=monitor)

print("Accuracy score (training): {0:.3f}".format(
    bdt.score(X_train, y_train)))
print("Accuracy score (validation): {0:.3f}".format(
    bdt.score(X_test, y_test)))

training
Step   1/100 - loss: 0.61762, acc: 0.86357 | ETA: 05:36
Step   2/100 - loss: 0.45651, acc: 0.88365 | ETA: 05:47
Step   3/100 - loss: 0.36652, acc: 0.89575 | ETA: 05:30
Step   4/100 - loss: 0.31619, acc: 0.90426 | ETA: 05:19
Step   5/100 - loss: 0.28669, acc: 0.91001 | ETA: 05:14
Step   6/100 - loss: 0.26394, acc: 0.91500 | ETA: 05:12
Step   7/100 - loss: 0.24829, acc: 0.91847 | ETA: 05:05
Step   8/100 - loss: 0.23533, acc: 0.92140 | ETA: 04:57
Step   9/100 - loss: 0.22656, acc: 0.92475 | ETA: 04:51
Step  10/100 - loss: 0.21873, acc: 0.92644 | ETA: 04:45
Step  11/100 - loss: 0.21359, acc: 0.92798 | ETA: 04:41
Step  12/100 - loss: 0.20827, acc: 0.92932 | ETA: 04:36
Step  13/100 - loss: 0.20397, acc: 0.93073 | ETA: 04:31
Step  14/100 - loss: 0.19983, acc: 0.93198 | ETA: 04:27
Step  15/100 - loss: 0.19640, acc: 0.93289 | ETA: 04:23
Step  16/100 - loss: 0.19358, acc: 0.93396 | ETA: 04:19
Step  17/100 - loss: 0.19048, acc: 0.93504 | ETA: 04:15
Step  18/100 - loss: 0.18783, acc: 0.93

In [17]:
print('Save the importance')
importances = bdt.feature_importances_
f = open('gbdt_results_9var/' + task_name + '/output_importance.txt', 'w')
f.write("%-35s%-15s\n" % ('Variable Name', 'Output Importance'))
for i in range(len(branch_names)):
    f.write("%-35s%-15s\n" % (branch_names[i], importances[i]))
    print("%-35s%-15s\n" % (branch_names[i], importances[i]), file=f)
f.close()

# y_predicted = bdt.predict(X_train)
y_predicted = bdt.predict(X_test)

Save the importance


In [18]:
# Save the BDT
# https://scikit-learn.org/stable/model_persistence.html

os.makedirs(model_dir, exist_ok=True)
save_path = f"{model_dir}/{task_name}_bdt.joblib"
print(f"Saving {task_name} model to {save_path}...")
joblib.dump(bdt, save_path)

Saving scalar1 model to /data/wifeng/photon-jet/bdt_models_v1.0/scalar1_bdt.joblib...


['/data/wifeng/photon-jet/bdt_models_v1.0/scalar1_bdt.joblib']